In [1]:
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


关闭警告

In [3]:
# 关闭警告
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

初体验

In [4]:
w = tf.Variable([[0.5,1.0]])
x = tf.Variable([[2.0],[1.0]])
y = tf.matmul(w,x)
y

<tf.Tensor 'MatMul:0' shape=(1, 1) dtype=float32>

## 图
图默认已经注册，  
包含 tf.Operation计算操作对象和tf.Tensor数据对象

In [5]:
# 如果你是使用 GPU 版 TensorFlow 的话，并且你想在显卡高占用率的情况下（比如玩游戏）训练模型，
# 那你要注意在初始化 Session 的时候为其分配固定数量的显存，否则可能会在开始训练的时候直接报错退出：
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [6]:
graph = tf.get_default_graph()
graph   # 定义变量之后，会自动生成一个图，相当于给程序分配内存

In [8]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(y))
    print(w.graph)
    print(sess.graph)
    print(y.graph)
    print(y)
    print(y.eval())  # eval把值拿出来

[[2.]]
Tensor("MatMul:0", shape=(1, 1), dtype=float32)
[[2.]]


图的创建

In [9]:
# 创建一个新的图
# 创建一张图包含了一组op和tensor，上下文环境
# op：只要使用tensorflow的APT定义的函数都是OP
# tensor：指的是数据
g = tf.Graph()
print(g)
with g.as_default():
    c = tf.constant(11.0)
    print(c.graph)

In [11]:
# 可指定运行图
with tf.Session(graph=g) as sess:
    print(sess.run(c))
    print(g)

11.0


Tensorflow Feed操作

In [12]:
import tensorflow as tf
input1 = tf.placeholder(tf.float32)   #  占位、占坑
input2 = tf.placeholder(tf.float32)
output = tf.add(input1,input2)
with tf.Session() as sess:
    print(sess.run([output],feed_dict={input1:[[10.0],[1]],input2:[[20.0],[1]]}))

[array([[30.],
       [ 2.]], dtype=float32)]


张量的属性：
  
* 张量所属的图，  
  
* 张量的name，  

* op(字符串描述)  

* 形状
* 张量对应的操作

In [13]:
a = tf.constant(5.0)
b = tf.constant(6.0)
plt = tf.placeholder(tf.float32, [3,3])
sum1 = tf.add(a, b)
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(plt, feed_dict={plt: [[1, 2, 3], [4, 5, 36], [2, 3, 4]]}))
    print(sess.run([a,b,sum1])) # 可以run多个变量，需放在列表里
    print(sum1.eval())  # eval() 显示数据的一种方式
    print("------------graph----------")
    print(a.graph)
    print("-----------shape-----------")
    print(a.shape)
    print(plt.shape)
    print("---------name--------------")
    print(a.name)
    print("-----------op--------------")
    print(plt.op)
# # tensorflow:打印出来的形状表示
# 0维：()   1维:(5)  2维：(5,6)   3维：(2,3,4)

[[ 1.  2.  3.]
 [ 4.  5. 36.]
 [ 2.  3.  4.]]
[5.0, 6.0, 11.0]
11.0
------------graph----------
-----------shape-----------
()
(3, 3)
---------name--------------
Const:0
-----------op--------------
name: "Placeholder_2"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 3
      }
      dim {
        size: 3
      }
    }
  }
}



非op不能运行，但如果运算中有tensor，将会把运算后的变量也设为op类型

In [17]:
a

<tf.Tensor 'Const:0' shape=() dtype=float32>

In [16]:
#不是op不能运行
var1 = 2.0
var2 = 3
sum2 = var1 + var2


# 有重载的机制,默认会给运算符重载成op类型
sum3 = a + var1

with tf.Session() as sess:
    # 可运行sum3但是不能运行sum2说明：如果运算中有tensor，
   #  print(sess.run(sum2)) 不能运行sum2,因为sum2这个操作并没有tensorflow里面的op（operation）
    print(sess.run(sum3))
    print(sess.run(a))
    print(a.eval())
    print(sum3.eval())

7.0
5.0
5.0
7.0


静态形状和动态形状

In [20]:
### 静态形状和动态形状
plt = tf.placeholder(tf.float32,[None,2])
print(plt)
print(plt.get_shape())
plt.set_shape([3,2])
#plt.set_shape([4,2]) # 静态形状已经设置固定，不能再次设置形状
print(plt)

plt_reshape = tf.reshape(plt,[2,3])
print(plt_reshape)

with tf.Session() as sess:
    pass

Tensor("Placeholder_5:0", shape=(?, 2), dtype=float32)
(?, 2)
Tensor("Placeholder_5:0", shape=(3, 2), dtype=float32)
Tensor("Reshape_2:0", shape=(2, 3), dtype=float32)


张量操作--生成张量（类似numpy）

In [21]:
import numpy as np
n = np.random.rand(2,3)
tf.constant(n)
n

array([[0.68991903, 0.1721874 , 0.45916735],
       [0.77700923, 0.48142376, 0.29444463]])

In [22]:
tf.zeros([3,4],"int32")

<tf.Tensor 'zeros:0' shape=(3, 4) dtype=int32>

In [23]:
z = tf.zeros([3,4])
ze = tf.zeros_like([3,5,2],'int32') # zero_like:仿照【3,5,2】的零张量
with tf.Session() as sess:
    print(sess.run(z))
    print(sess.run(ze))

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[0 0 0]


In [26]:
tf.ones([2,3],"int32")

<tf.Tensor 'ones:0' shape=(2, 3) dtype=int32>

In [27]:
tf.constant([1,2,3,5])

<tf.Tensor 'Const_7:0' shape=(4,) dtype=int32>

In [24]:
tf.constant(-1,shape=[2,3])

<tf.Tensor 'Const_3:0' shape=(2, 3) dtype=int32>

In [25]:
lins = tf.linspace(10.0,12.0,3,name="linspace") # 前两个参数要写成浮点数形式，不然 会报错
lins

<tf.Tensor 'linspace:0' shape=(3,) dtype=float32>

张量--类型转换

In [26]:
## 类型转换
x = tf.constant(1.0)
y = tf.cast(x,tf.int32)
y = tf.cast(x,'int32')
y

<tf.Tensor 'Cast_1:0' shape=() dtype=int32>

使用交互模型

In [27]:
tf.InteractiveSession()  # 使用交互模式
a = [1,2]
b = [3,4]
c = tf.concat([a,b],axis=0)
c.eval()
# e = tf.Variable([2.0])
# e.eval()

array([1, 2, 3, 4])

In [28]:
sess = tf.Session()
type(sess.run(tf.range(1,6,2)))

numpy.ndarray

In [29]:
tf.range(1,6,2).eval()

array([1, 3, 5])

生成随机数

In [30]:
norm = tf.random_normal([2,3],mean=-1,stddev=4)
c = tf.constant([[1,2],[3,4],[5,6]])
shuff = tf.random_shuffle(c)
sess = tf.Session()
print(sess.run(norm))
print(sess.run(shuff))

[[ 0.7335875  -1.7162688   6.9096274 ]
 [ 0.41651773  0.67861736  2.2102523 ]]
[[1 2]
 [5 6]
 [3 4]]


In [31]:
sess.run(lins)

array([10., 11., 12.], dtype=float32)

In [32]:
with tf.Session() as sess:
    print(sess.run(lins))

[10. 11. 12.]


变量的创建，变量的初始化  
变量不初始化会报错

In [35]:
state = tf.Variable(0)
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value) # 把后面的值赋给前面

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # 运行变量之前，要添加一个全局变量初始化并运行
    #print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

1
2
3


会话（变量）的保存

In [36]:
import tensorflow as tf

In [37]:
tf.train.Saver
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run([w1,w2]))
print(w1.op)
saver.save(sess, './save/test')

[array([-0.6161127, -1.189865 ], dtype=float32), array([ 0.84259087,  1.8901141 , -0.13243528,  0.11345746,  0.31676388],
      dtype=float32)]
name: "w1"
op: "VariableV2"
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 2
      }
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: ""
  }
}



'./save/test'

In [42]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph("./save/test.meta")
    saver.restore(sess,tf.train.latest_checkpoint("./save"))
    print(sess.run("w1:0"))

INFO:tensorflow:Restoring parameters from ./save\test
[-0.6161127 -1.189865 ]


tensorboard可视化  
tensorboard --logdir=/summary/test/

In [1]:
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Tenboard可视化
w = tf.Variable([[0.5,1.0]],name='w')
x = tf.Variable([[2.0],[1.0]],name='x')
y = tf.matmul(w, x)
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(y))
    file_writer = tf.summary.FileWriter('./save/test1', graph=sess.graph)

[[2.]]


numpy和tensor转换

In [3]:
import numpy as np
a = np.zeros((3,3))
ta = tf.convert_to_tensor(a)
tb = tf.constant(a)
#init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    #print(sess.run(init_op))
    print(sess.run(ta))
    print(sess.run(tb))
    print(tb.eval())
    print(ta.eval())

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


placeholder占坑操作

In [4]:
# placeholder 占坑操作，占位
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1,input2)
with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7,1], input2:[2,2]}))

[array([14.,  2.], dtype=float32)]


In [8]:
# placeholder 占坑操作，占位
# 乘法有元素级的乘法multiply，和矩阵乘法matmul
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.matmul(input1,input2)
with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[[1],[2]], input2:[[2,2]]}))

[array([[2., 2.],
       [4., 4.]], dtype=float32)]


总结，如果声明的变量即使用了Variable(),后面一定要有global_variable_initializer()  

没有用Variable时，可以不用，只需建立会话，运行即可